# Práctica 3: Representación del conocimiento

__Fecha de entrega: 20 de febrero de 2023__

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de Representación de conocimiento. La práctica consta de un único notebook que se entregará en la tarea de entrega habilitada en el Campus  Virtual.

Cada consulta debe contener breves comentarios que expliquen cada tripleta. __No se valorarán consultas sin explicaciones__.

__Número de grupo: 12__

__Nombres de los estudiantes: Greta Angolani y Héloïse Lafargue__

## Consultas SPARQL sobre Wikidata.

En esta práctica vamos a usar el [punto de acceso SPARQL](https://query.wikidata.org/) de Wikidata para contestar las preguntas que se formulan a continuación. Cada pregunta debe ser respondida realizando una única consulta SPARQL. Para cada una de las entidades recuperadas se mostrará __tanto su identificador como su etiqueta__ (nombre de la entidad en lenguaje natural). 

Para cada una de las preguntas debes mostrar tanto la consulta como la respuesta obtenida. Si lo consideras necesario, puedes añadir celdas adicionales en formato _Markdown_ para explicar decisiones que hayas tomado al crear la consulta o cualquier otro dato que consideres interesante.
 
Para resolver estas consultas puedes usar __dos recursos que te recomendamos consultar__ son:

- [Este tutorial de SPARQL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial).
- [Esta recopilación de ejemplos](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

In [2]:
## Lo primero sería instalar en tu entorno de Python wdsparql
## Si no lo tienes instalado, puedes hacerlo desde el notebook
import sys

!{sys.executable} -m pip install wdsparql

# Asumismos instalado wdsparql
%load_ext autoreload
%load_ext wdsparql
%autoreload 2

### Ejemplo

Recuperar todas las instancias directas de la clase [Cabra (Q2934)](https://www.wikidata.org/wiki/Q2934) que aparecen en la base de conocimiento.

In [2]:
%%wdsparql
SELECT ?item ?itemLabel 
WHERE 
{
  ?item wdt:P31 wd:Q2934.  # instancias directas de la clase Cabra
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,item,itemLabel
0,http://www.wikidata.org/entity/Q151345,Billygoat Hennes
1,http://www.wikidata.org/entity/Q3569037,William Windsor
2,http://www.wikidata.org/entity/Q13050139,Q13050139
3,http://www.wikidata.org/entity/Q23003932,His Whiskers
4,http://www.wikidata.org/entity/Q24287064,Taffy
5,http://www.wikidata.org/entity/Q41239734,Lance Corporal Shenkin III
6,http://www.wikidata.org/entity/Q41240892,Lance Corporal Shenkin II
7,http://www.wikidata.org/entity/Q41241416,Lance Corporal Shenkin I


### Consulta 1

[U2 (Q396)](https://www.wikidata.org/wiki/Q396) es una banda de rock con una larga trayectoria y numerosos éxitos. 

Vamos a comenzar por averiguar su fecha y lugar de formación (localidad y país).

In [3]:
%%wdsparql
SELECT (YEAR(?inception) AS ?year) ?locationLabel  # seleciona el ano de la fecha (es mas interesante porque contiene solo el ano)
WHERE 
{ 
    # consulta de las instancias de U2: wd:Q396
 wd:Q396 wdt:P571 ?inception. # la fecha de formacion es identificada por "inception" (P571) en la banda
 wd:Q396 wdt:P740 ?location. # el lugar de formacion de la banda es identificado por "location of formation" (P740)
 SERVICE wikibase:label { bd:serviceParam wikibase:language "en".}
}

,year,locationLabel
0,1976,Dublin


### Consulta 2

A continuación vamos a averiguar todos los miembros que se le conocen en la base de conocimiento. Queremos obtener los resultados ordenados alfabéticamente por su apellido e incluir su nombre completo y el artístico.

Puede que la siguiente información sobre el uso de [OPTIONAL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#OPTIONAL) te resulte útil.

In [4]:
%%wdsparql
SELECT ?memberLabel ?familyNameLabel ?pseudonymLabel
WHERE {
        # consulta de las instancias de U2: wd:Q396
    wd:Q396 wdt:P527 ?member. # los miembros de U2 estan identificados por la instancia "has part(s)" (P527)
    
    # A parameter is considered optional if the parameter has a default value specified when it is declared.
    # It is not necessary to provide a value for an optional parameter in a procedure call.
    # The default value of a parameter is used when: No value for the parameter is specified in the procedure call.
    OPTIONAL { ?member wdt:P742 ?pseudonym.} # dar el nombre artistico si existe
    OPTIONAL { ?member wdt:P735 ?givenName.} # dar el nombre si existe
    OPTIONAL { ?member wdt:P734 ?familyName.} # dar el apellido si existe
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?member ?familyNameLabel ?pseudonymLabel ?memberLabel # 
ORDER BY ?familyNameLabel # ordenar alfabéticamente por su apellido

,memberLabel,familyNameLabel,pseudonymLabel
0,Adam Clayton,Clayton,None
1,The Edge,Evans,The Edge
2,Dik Evans,Evans,None
3,Bono,Hewson,Bono
4,Larry Mullen Jr.,Mullen,None


Es importante usar dos OPTIONAL distintos para que se muestren todos los miembros, tengan o no pseudónimo y nombre completo asociados.

### Consulta 3

[Bono (Q834621)](http://www.wikidata.org/entity/Q834621) es el cantante de la banda. Además de cantante, ha tenido muchas otras ocupaciones. ¿Cuáles de ellas están relacionadas directamente con el mundo del [arte (Q483501](https://www.wikidata.org/wiki/Q483501)? Ten en cuenta que la jerarquía de ocupaciones puede ser muy sofisticada.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Property_paths) te resulte útil.

In [5]:
%%wdsparql
SELECT ?occupationLabel 
WHERE {
    # consultas de Bono : wd:Q834621
  wd:Q834621 wdt:P106 ?occupation. # las occupaciones estan identificadas por P106
  ?occupation wdt:P279* wd:Q483501. # seleccionar las occupaciones relacionadas con la subclass (P279) del arte (Q483501)
    # "*" después de un "path" significa 0 o más de este elemento
  SERVICE wikibase:label {bd:serviceParam wikibase:language "en".}
}

,occupationLabel
0,screenwriter
1,actor
2,singer-songwriter
3,guitarist


Si no añadimos el `+` a la relación `wdt:P279` muchas de las profesiones no se recuperan.

### Consulta 4

Bono se ha casado una vez, con Ali Hewson, con la que tiene varios hijos. Queremos conocer los nombres de esos hijos y sus fechas de nacimiento. Los resultados deben aparecer ordenados cronológicamente.

In [6]:
%%wdsparql
SELECT ?childLabel ?birthDate
WHERE {
        # consultas de Bono : wd:Q834621
  wd:Q834621 wdt:P40 ?child. # los hijos estan en la instancia "child" (P40)
  ?child wdt:P569 ?birthDate. # seleccionar las fechas de nacimiento (P569) para cada hijo
  SERVICE wikibase:label {bd:serviceParam wikibase:language "en".}
}
ORDER BY ?birthDate # ordenadar cronológicamente

,childLabel,birthDate
0,Eve Hewson,1991-07-07T00:00:00Z
1,Elijah Hewson,1999-08-18T00:00:00Z


### Consulta 5

Bono ha recibido muchos premios a lo largo de su carrera. Queremos obtener la lista de premios y, para cada uno de ellos, la fecha en la que fue premiado (si está disponible). Los resultados se deben mostrar ordenado por fecha.

Para resolver esta consulta necesitarás acceder a los cualificadores de nodos sentencia y necesitarás entender los prefijos que usa Wikidata. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Qualifiers) te resulte útil.

In [7]:
%%wdsparql
SELECT ?awardLabel ?date
WHERE {
         # consultas de Bono : wd:Q834621
  wd:Q834621 p:P166 ?awardStatement. # los premios de Bono estan en la instancia "award received" (P166)
  ?awardStatement ps:P166 ?award.  # qualifier award
  OPTIONAL {?awardStatement pq:P585 ?date.}  # para seleccionar la fecha si está disponible necesitamos usar OPTIONAL
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en".}
}
ORDER BY ?date

,awardLabel,date
0,Grammy Awards,None
1,Fellow of the American Academy of Arts and Sci...,None
2,MusiCares Person of the Year,2003-01-01T00:00:00Z
3,Knight of the Legion of Honour,2003-01-01T00:00:00Z
4,TED Prize,2005-01-01T00:00:00Z
5,Rock and Roll Hall of Fame,2005-01-01T00:00:00Z
6,Officer of the Order of Liberty,2005-04-21T00:00:00Z
7,Time Person of the Year,2005-12-19T00:00:00Z
8,Philadelphia Liberty Medal,2007-01-01T00:00:00Z
9,NAACP Image Award – Chairman's Award,2007-01-01T00:00:00Z


### Consulta 6

Ahora queremos conocer todos los discos de U2 del tipo [albunes en directo (Q209939)](https://www.wikidata.org/wiki/Q209939) y cualquiera de sus subgéneros. Los resultados se deben mostrar ordenados por nombre e incluir el año de publicación.

In [53]:
%%wdsparql
SELECT ?studio_albLabel ?year (group_concat(?subgenreLabel;separator="/") as ?subgenreLabels) {
    # we use the group_concat to concatenate the subgenres when there is more than one, to not repeat the studio_albLabel
    wd:Q396 wdt:P358 ?disco. # discography U2 in "discography" (P358) from U2 (Q396)
    ?disco wdt:P2354 ?album. # in this discography we want the list of album: "has list" (P2354)
    ?album wdt:P527* ?studio_alb. # we want the albums that "has part of" (P527) of this discography
    ?studio_alb wdt:P7937 wd:Q209939. # in this albums, we want the ones "form of creative work" (P7937) is the "live albums" (Q209939)
    OPTIONAL { ?studio_alb wdt:P136 ?subgenre} # have the subgenre if it exits
    ?studio_alb wdt:P577 ?date. # have the date
    BIND(YEAR(?date) AS ?year) # converting date in year format
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". 
                           ?studio_alb rdfs:label ?studio_albLabel .
                           ?subgenre rdfs:label ?subgenreLabel .}
}
GROUP BY ?studio_albLabel ?year # for the concatenation of subgenreLabel
ORDER BY ?studio_albLabel

,studio_albLabel,year,subgenreLabels
0,Rattle and Hum,1988,blues rock/roots rock


### Consulta 7

Queremos saber si hay obras de U2 publicadas varias veces (en diferentes años). Para ello vamos a recuperar las obras de tipo [lanzamiento de un grupo (group release)](https://www.wikidata.org/wiki/Q108346082) pero mostrando también la fecha de publicación más antigua de cada una y el número de apariciones. Los resultados deben aparecer en orden cronológico.

Para calcular la fecha correcta tendrás que agrupar las respuestas por album y aplicar una función de agregación sobre las fechas de lanzamiento. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Grouping) te resulte útil.

In [9]:
%%wdsparql
SELECT ?songLabel (MIN(?date) AS ?oldestDate) (COUNT(?song) AS ?numAppearances)
WHERE {
    wd:Q396 wdt:P358 ?disco. # discography U2 in "discography" (P358) from U2 (Q396)
    ?disco wdt:P2354 ?album. # in this discography we want the list of album: "has list" (P2354)
    ?album wdt:P527* ?tracklist. # we want the albums that "has part of" (P527) of this discography
    ?tracklist wdt:P658 ?song. # in this albums, we want the ones which have a " tracklist" (P658) so we have the songs
    ?song wdt:P577 ?date. # the date
    BIND(YEAR(?date) AS ?year) # converting date in year format
    OPTIONAL { ?song wdt:P577 ?date. } # Obtener la fecha de lanzamiento
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?songLabel
HAVING (COUNT(?song) > 1) # only select albums that appear more than once
ORDER BY ?date

,songLabel,oldestDate,numAppearances
0,Out of Control,1979-01-01T00:00:00Z,2
1,Stories for Boys,1979-01-01T00:00:00Z,2


### Consulta 8

U2 es sin duda un grupo con una larga trayectoria. ¿Cuántas discos ha lanzado? Vamos a considerar sólo instancias directas de [album de estudio (Q208569)](https://www.wikidata.org/wiki/Q208569).

In [10]:
%%wdsparql
SELECT (COUNT(DISTINCT ?studio_alb) AS ?count)
WHERE {
    wd:Q396 wdt:P358 ?disco. # discography U2 in "discography" (P358) from U2 (Q396)
    ?disco wdt:P2354 ?album. # in this discography we want the list of album: "has list" (P2354)
    ?album wdt:P527* ?studio_alb. # we want the albums that "has part of" (P527) of this discography
    ?studio_alb wdt:P7937 wd:Q208569. # in this albums, we want the ones "form of creative work" (P7937) is the "studio album" (Q208569) 
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,count
0,4


### Consulta 9

Entre todos sus discos de estudio, queremos recuperar los títulos, fechas de lanzamiento y discográfica de los discos de la década de los 1990. Si alguna obra tiene varias fechas de publicación, se mostrará si alguna de ellas está dentro de ese periodo.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#FILTER) te resulte útil.

In [1]:
%%wdsparql
SELECT DISTINCT ?studio_albLabel ?date ?labelLabel
WHERE {
    wd:Q396 wdt:P358 ?disco. # discography U2 in "discography" (P358) from U2 (Q396)
    ?disco wdt:P2354 ?album. # in this discography we want the list of album: "has list" (P2354)
    ?album wdt:P527* ?studio_alb. # we want the albums that "has part of" (P527) of this discography
    ?studio_alb wdt:P7937 wd:Q208569. # in this albums, we want the ones "form of creative work" (P7937) is the "studio album" (Q208569) 
    ?studio_alb wdt:P577 ?date. # the date (P577) of the studio album
    ?studio_alb wdt:P264 ?label. # the label (P264) of the studio album
    FILTER(YEAR(?date) >= 1990 && YEAR(?date) <= 1999). # filter the date on the decade 1990
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?date
# there is no U2's studio album published in the 1990 decade.

UsageError: Cell magic `%%wdsparql` not found.


### Consulta 10

¿Qué otros personajes famosos nacieron el mismo año que Bono y tiene su misma nacionalidad? Para cada uno de ellos muestra su nombre y ocupación. Muestra los resultados ordenados por nombre y fecha de nacimiento.

In [40]:
%%wdsparql
SELECT ?personLabel ?dateOfBirth (group_concat(?occupationLabel;separator="/") as ?occupationLabels) {
    # we use the group_concat to concatenate the occupations when there is more than one, to not repeat the personLabel

    #Get Bono's date of birth and nationality
    wd:Q834621 wdt:P569 ?bonoBirth.
    wd:Q834621 wdt:P31 ?bonoNationality.
    #Get all people born in the same year and same country of citizenship (Irish) as Bono
    ?person wdt:P569 ?dateOfBirth.
    ?person wdt:P31 ?nationality.
    ?person wdt:P27 wd:Q27.
    FILTER(YEAR(?dateOfBirth) = YEAR(?bonoBirth))
    FILTER(?nationality = ?bonoNationality)
    #Get the occupation of each person
    OPTIONAL { ?person wdt:P106 ?occupation }
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". 
                            ?person rdfs:label ?personLabel .
                           ?occupation rdfs:label ?occupationLabel .}
}
GROUP BY ?personLabel ?dateOfBirth # for the occupation concatenation
ORDER BY ?personLabel ?dateOfBirth

,personLabel,dateOfBirth,occupationLabels
0,Alan Campbell,1960-08-10T00:00:00Z,association football player
1,Alan Glynn,1960-01-01T00:00:00Z,screenwriter/writer/journalist/science fiction...
2,Alan Moore,1960-02-02T00:00:00Z,poet
3,Amira Ghazalla,1960-01-01T00:00:00Z,actor/stage actor/film actor
4,Andrew Doyle,1960-07-02T00:00:00Z,politician/farmer
5,Ann Hawkshaw,1960-01-01T00:00:00Z,badminton player
6,Ann O'Sullivan,1960-01-01T00:00:00Z,badminton player
7,"Arthur Doe, Jr.",1960-01-01T00:00:00Z,
8,Austin Buckley,1960-01-01T00:00:00Z,hurler
9,Bernie Nolan,1960-10-17T00:00:00Z,actor/singer/autobiographer


### Consulta 11

¿Cuántos asertos hay sobre U2 en Wikidata? Ten en cuenta que puede aparece tanto como sujeto como objeto de cada tripleta.

In [14]:
%%wdsparql
SELECT (COUNT(*) AS ?count) 
WHERE {
  { wd:Q396 ?p ?o } # U2 is the subject
  UNION # we take in count both possibilities in the query
  { ?s ?p wd:Q396 } # U2 is the object
}

,count
0,1427
